Suppose our data are IID normal, $x_i \sim \cal{N}(\mu, \sigma)$, so the likelihood is

$$
p(x\,|\,\mu, \sigma) \propto \prod_{i=1}^n \exp{\bigg( -\frac{(x_i - \mu)^2}{2\sigma^2}\bigg)}
$$


Suppose further that $\sigma$ is known and the prior for $\mu$ is $\cal{N}(\mu_0, \sigma_0)$, that is
$$
\pi(\mu) \propto \exp{\bigg( -\frac{(\mu - \mu_0)^2}{2\sigma_0^2}\bigg)}
$$

We can calculate the posterior analytically

$$
\begin{aligned}
p(\mu\,|\, \boldsymbol{x}) &\propto \exp{\bigg(
-\frac{(\mu - \mu_0)^2}{2\sigma_0^2}
- \frac{\sum_{i=1}^n(x_i - \mu)^2}{2\sigma^2}\bigg)} \\
&\propto \exp{\bigg[-\frac{1}{2}\bigg(\frac{\mu^2 \sigma^2 -2\sigma^2\mu\mu_0 - 2\sigma_0^2n\bar{x}\mu + \sigma_0^2 n\mu^2}{\sigma^2\sigma_0^2}\bigg)\bigg]} \\
&= \exp{\bigg[-\frac{1}{2}\bigg(\frac{ (n\sigma_0^2 + \sigma^2)\mu^2 - 2(\sigma^2\mu_0 - \sigma_0^2n\bar{x})\mu}{\sigma^2\sigma_0^2}\bigg)\bigg]} \\
&= \exp{\Bigg[-\frac{1}{2}\Bigg(\frac{ \mu^2 - 2\mu\frac{(\sigma^2\mu_0 - \sigma_0^2n\bar{x})}{(n\sigma_0^2 + \sigma^2)}}{\frac{\sigma^2\sigma_0^2}{(n\sigma_0^2 + \sigma^2)}}\Bigg)\Bigg]} \\
&\propto \exp{\Bigg[-\frac{1}{2}\Bigg(\frac{\big(\mu - \frac{(\sigma^2\mu_0 - \sigma_0^2n\bar{x})}{(n\sigma_0^2 + \sigma^2)}\big)^2}{\frac{\sigma^2\sigma_0^2}{(n\sigma_0^2 + \sigma^2)}}\Bigg)\Bigg]}
\end{aligned}
$$

In other words

$$
\mu\,|\, \boldsymbol{x} \sim \cal{N}\bigg(\frac{\sigma^2\mu_0 + n\sigma_0^2\bar{x}}{n\sigma_0^2 + \sigma^2}, \frac{\sigma^2\sigma_0^2}{n\sigma_0y^2 + \sigma^2} \bigg)
$$


## Markov Chain Monte Carlo

Let's create some fake data.

In [1]:
import qualified Data.Vector.Unboxed as V
import Data.Random.Source.PureMT
import Data.Random
import Control.Monad.State
import Data.Histogram ( asList )
import Data.Histogram.Fill
import Data.Histogram.Generic ( Histogram )

In [2]:
nSamples, seed :: Int
nSamples = 10
seed = 2

In [3]:
mu0, sigma0, sigma :: Double
mu0 = 11.0
sigma0 = 2.0
sigma = 1.0

In [4]:
d :: Double
d = 2.0

In [5]:
simpleXs :: [Double]
simpleXs =
  evalState (replicateM nSamples hierarchicalSample)
            (pureMT $ fromIntegral seed)
  where
    hierarchicalSample = do
      mu <- sample (Normal mu0 sigma0)
      sample (Normal mu sigma)

In [6]:
normalisedProposals :: Int -> Double -> Int -> [Double]
normalisedProposals seed sigma nIters =
  evalState (replicateM nIters (sample (Normal 0.0 sigma)))
  (pureMT $ fromIntegral seed)

acceptOrRejects :: Int -> Int -> [Double]
acceptOrRejects seed nIters =
  evalState (replicateM nIters (sample stdUniform))
  (pureMT $ fromIntegral seed)

prior :: Double -> Double
prior mu = exp (-(mu - mu0)**2 / (2 * sigma0**2))

likelihood :: Double -> [Double] -> Double
likelihood mu xs = exp (-sum (map (\x -> (x - mu)**2 / (2 * sigma**2)) xs))

posterior :: Double -> [Double] -> Double
posterior mu xs = likelihood mu xs * prior mu

acceptanceProb :: Double -> Double -> [Double] -> Double
acceptanceProb mu mu' xs = min 1.0 (posterior mu' xs / posterior mu xs)

oneStep :: (Double, Int) -> (Double, Double) -> (Double, Int)
oneStep (mu, nAccs) (proposedJump, acceptOrReject) =
  if acceptOrReject < acceptanceProb mu (mu + proposedJump) simpleXs
  then (mu + proposedJump, nAccs + 1)
  else (mu, nAccs)

In [7]:
test :: [(Double, Int)]
test = drop 200000 $
       scanl oneStep (10.0, 0) $
       zip (normalisedProposals 3 0.4 300000) (acceptOrRejects 4 300000)

In [8]:
hb :: HBuilder Double (Data.Histogram.Generic.Histogram V.Vector BinD Double)
hb = forceDouble -<< mkSimple (binD (10.0 - 1.5*sigma0) 400 (10.0 + 1.5*sigma0))

hist :: Histogram V.Vector BinD Double
hist = fillBuilder hb (map fst test)

In [9]:
:set -fno-ghci-sandbox

In [10]:
{-# LANGUAGE TemplateHaskell #-}
{-# LANGUAGE QuasiQuotes #-}

import qualified Language.R as R
import Language.R.QQ

Line 1: Unused LANGUAGE pragma
Found:
{-# LANGUAGE TemplateHaskell #-}
Why not:
Line 2: Unused LANGUAGE pragma
Found:
{-# LANGUAGE QuasiQuotes #-}
Why not:

In [11]:
xs = map fst $ asList hist
ys = map snd $ asList hist

In [12]:
  R.runRegion $ do
    [r| library(ggplot2) |]
    df <- [r| data <- data.frame(xs_hs, ys_hs) |]
    p1 <- [r| ggplot(df_hs, aes(x=xs_hs, y = ys_hs)) + geom_histogram(stat="identity") |]
    [r| ggsave(filename="diagrams/TestHist.pdf") |]
    return ()

Saving 7 x 7 in image